#  STEP 4: Variant Calling (Phase 6 R&D)

**Goal:** Implement our scientific decision to filter the cohort, and then develop the "recipe" for Variant Calling.

**Why:**
1.  **Filtering (The "Decision"):** In `Notebook 04`, we identified 3 low-quality outlier samples (`PA033`, `PA045`, `PA070`). We MUST remove these from our analysis pipeline *before* running Variant Calling. We will create a new "clean" metadata file (`metadata_final_cohort.csv`) containing only the 93 high-quality samples.
2.  **R&D (The "Recipe"):** We need to test the `bcftools` command "recipe" (specifically `bcftools mpileup | bcftools call`) on a *single* sample to prove it works before automating it in the `Snakefile`.

In [ ]:
import pandas as pd
import os

print("--- STEP 1: Implementing the Scientific Decision (Filtering) ---")

# --- 1. Define Paths (CORRECTED: No "../") ---
# (We are running from the project root)
file_in = "results/metadata/metadata_clean.csv"
file_out = "results/metadata/metadata_final_cohort.csv"

# --- 2. Define the Outliers (From Notebook 04) ---
outliers_to_remove = ['PA033', 'PA045', 'PA070']

print(f"Loading {file_in} (which has 96 samples)...")
df = pd.read_csv(file_in)

# --- 3. The Filter (The "Decision") ---
# We use .isin() to find all rows *NOT* (~) in our outlier list
df_clean = df[~df['sample_id'].isin(outliers_to_remove)]

# --- 4. Verification (Rule 1) ---
print(f"Removed {len(outliers_to_remove)} samples.")
print(f"New cohort size: {len(df_clean)} samples (Expected: 93).")

# --- 5. Save the new "Brain" (The Handoff) ---
df_clean.to_csv(file_out, index=False)
print(f"\nSUCCESS: Saved new 'metadata_final_cohort.csv' with 93 samples.")

##  1: R&D Test (Variant Calling "Recipe")

**Goal:** Test the full Variant Calling "pipe" on a *single sample*.

**Why (Rule 2: R&D Lab):**
Before we automate 93 samples, we must test the "recipe" (the command). This is a 2-step "pipe":
1.  **`bcftools mpileup`:** Reads the reference (`.fna`) and the sorted BAM (`.bam`) to "pile up" all base calls at every position.
2.  **`bcftools call`:** Reads the "pileup" data and makes a statistical "call" to find *only* the positions that are different from the reference (the Variants/SNPs).

**R&D Test:** We will pipe (`|`) these two commands and save the result as a compressed VCF (`.vcf.gz`) for our test sample, `PA001`.

In [ ]:
import os

print("--- STEP 2: Testing the bcftools 'mpileup | call' pipe ---")

# --- 1. Define Paths (Relative to project root) ---
ref_genome = "data/reference_genome/PAO1_reference.fna"

# We'll use our trusty "Test 1" sample
input_bam = "results/mapped_reads/PA001.sorted.bam"

# This is a *NEW* results directory (Rule 4)
output_dir = "results/variant_calling"
os.makedirs(output_dir, exist_ok=True)

# The final VCF file for our test sample
# We use .vcf.gz because VCFs are huge text files
output_vcf = f"{output_dir}/TEST_PA001.vcf.gz"

# --- 2. Build the Pipe Command (Test 2) ---
# -f : Reference FASTA
# -m : "multiallelic-caller" (Good for bacteria)
# -v : "variants-only" (CRITICAL: only report SNPs)
# -O z : Output compressed VCF (.gz)
# -o : Output file path
command = f"""
bcftools mpileup -f {ref_genome} {input_bam} \
    | bcftools call -mv -O z -o {output_vcf} -
"""

print(f"Starting Variant Calling pipe for {input_bam}...")
print("(This may take a minute or two)...")

# --- 3. Run the R&D Test ---
!{command}

print("\nPipe complete.")

# --- 4. Verification (Rule 1) ---
print(f"\n--- Verification: Checking for final VCF file ---")
!ls -lh {output_vcf}

##  3: R&D Correction (Haploid vs Diploid)

**Analysis:**
The R&D Test in `Cell 4` was a *technical* success (it produced a `.vcf.gz` file), but a *scientific* failure.

**The "Red Flag":**
The log clearly showed the warning: `assuming all sites are diploid`.

**The "Why" (Rule 3):**
This is the default assumption for human genomics. However, our bacteria (`P. aeruginosa`) are **Haploid** (1n), not Diploid (2n). Running with the wrong "ploidy" setting will cause the `bcftools call` algorithm to make incorrect statistical calls (e.g., misinterpreting noise as heterozygous SNPs).

**The "Fix" (The Clean Protocol - Rule 4):**
We must re-run the test using the correct flag for haploid organisms: `--ploidy 1`.

In [ ]:
import os

print("--- STEP 3: Testing the CORRECTED (Haploid) 'mpileup | call' pipe ---")

# --- 1. Define Paths (No changes) ---
ref_genome = "data/reference_genome/PAO1_reference.fna"
input_bam = "results/mapped_reads/PA001.sorted.bam"
output_dir = "results/variant_calling"

# We will OVERWRITE the old "TEST" file with the correct one
output_vcf = f"{output_dir}/TEST_PA001.vcf.gz"

# --- 2. Build the CORRECTED Pipe Command (The "Fix") ---
# -f : Reference FASTA
# -m : "multiallelic-caller"
# -v : "variants-only"
# -O z : Output compressed VCF (.gz)
# -o : Output file path
# --- [THE FIX] ---
# --ploidy 1 : We are telling bcftools this is HAPLOID data.
command = f"""
bcftools mpileup -f {ref_genome} {input_bam} \
    | bcftools call --ploidy 1 -mv -O z -o {output_vcf} -
"""

print(f"Starting Variant Calling pipe (Haploid) for {input_bam}...")
print("(This should take the same amount of time)...")

# --- 3. Run the Corrected R&D Test ---
!{command}

print("\nPipe complete.")

# --- 4. Verification (Rule 1) ---
print(f"\n--- Verification: Checking for final (Haploid) VCF file ---")
!ls -lh {output_vcf}

## Conclusion & Handoff to "The Factory"

**Status:** Success. The R&D for Phase 6 is 100% complete.

**Our Achievements (The "Recipe"):**
1.  **Filtered Cohort:** We executed our scientific decision (from `Notebook 04`) and created `results/metadata/metadata_final_cohort.csv`, which contains only our 93 high-quality samples. This is the new "brain" for the pipeline.
2.  **Corrected Recipe:** We discovered the critical `diploid` vs `haploid` error. We have now proven that the **correct (Haploid)** "recipe" for variant calling is:
    `bcftools mpileup -f [REF] [BAM] | bcftools call --ploidy 1 -mv -O z -o [VCF] -`

**Next Step (The Handoff):**
This notebook (`05_...ipynb`) is now complete. We will save it, and our new metadata file, to GitHub (Rule 5).

We are ready to automate this "recipe" in our main `Snakefile` to run it on all **93 clean samples**.